In [1]:
import cobra
import framed
import cobamp
import pandas as pd
import numpy as np
import pickle
import scipy as sci



In [2]:
from cobamp.wrappers import MatFormatReader
from cobamp.wrappers import COBRAModelObjectReader
from troppo.methods.reconstruction.imat import IMAT
from troppo.methods.reconstruction.imat import IMATProperties
from troppo.utilities.statistics import normalize, z_score

/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(


In [3]:
def load_glial_data(path):
    data = pd.read_csv(path, index_col=0)
    normalized_data = normalize(data)
    z = None

    return normalized_data, z

def get_index_core(reactions_ids, data, threshold):
    rx_names = data.index[data['0'] > threshold].tolist()
    index_core = [np.where(reactions_ids == r)[0][0] for r in rx_names]
    return index_core, rx_names

def get_index_values():
    pass

In [4]:
mat = sci.io.loadmat('/home/jiang817/cybergut_project/General_models/Recon3D.mat')['Recon3D']


In [5]:
model_cobra = cobra.io.load_matlab_model('/home/jiang817/cybergut_project/General_models/Recon3D.mat')

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 921650
Academic license - for non-commercial use only - registered to jiang817@purdue.edu


No defined compartments in model Recon3D. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [48]:
model_cobra

Name,model
Memory address,14ee48bae5b0
Number of metabolites,3934
Number of reactions,6361
Number of genes,1954
Number of groups,1
Objective expression,1.0*BIOMASS_maintenance - 1.0*BIOMASS_maintenance_reverse_5b3f9
Compartments,"m, r, c, e, l, g, x, n, i"


In [6]:
model = MatFormatReader(mat)
rx_names = model.get_reaction_and_metabolite_ids()[0]

for i in range(len(model.bounds_dict)):
    if rx_names[i].startswith('EX_'):
        model.bounds_dict[rx_names[i]] = (-100.0, 100.0)
        
S = model.S
lb, ub = model.get_model_bounds(False, True)

/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _1591_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _8639_AT1 or _26_AT1 or _314_AT2 or _314_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _1594_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _10993_AT1
  warnings.warn('Could not normalize this rule: ' + rule)
/home/jiang817/anaconda3/envs/BaloniLab/lib/python3.8/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _6818_AT1
  warni

In [37]:
exp_data = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_iMAT_INPUT/rxn_score_latest.csv')

data_dict = {}

for i in range(len(exp_data)):
    data_dict[exp_data.iloc[i, 0]] = exp_data.iloc[i, 1]

data_dict

{'24_25DHVITD3tm': nan,
 '25HVITD3t': nan,
 'COAtl': nan,
 'EX_5adtststerone_e': nan,
 'EX_5adtststerones_e': nan,
 'EX_5fthf_e': nan,
 'EX_5htrp_e': nan,
 'EX_5mthf_e': nan,
 'EX_5thf_e': nan,
 'EX_6dhf_e': nan,
 '24_25VITD3Hm': nan,
 '24NPHte': nan,
 '10FTHF7GLUtl': nan,
 '10FTHFtm': nan,
 '11DOCRTSLtr': nan,
 '13DAMPPOX': 0.0,
 '24_25DHVITD2t': nan,
 '24_25DHVITD2tm': nan,
 '24_25DHVITD3t': nan,
 '25VITD2Hm': nan,
 '2AMACHYD': nan,
 '2AMACSULT': nan,
 '2AMADPTm': nan,
 '2MCITt': nan,
 '2OXOADOXm': 10.39725944,
 '2OXOADPTm': nan,
 '34DHPHEt': nan,
 '34DHXMANDACOX': 0.0,
 '34HPLFM': nan,
 '3AIBtm': 10.80847916,
 '3HAO': 0.0,
 '3MOBt2im': nan,
 '3MOPt2im': nan,
 '3MOXTYROX': 11.92817258,
 '3SALAOX': nan,
 '42A12BOOX': 11.92817258,
 '4HBZCOAFm': nan,
 '4HBZFm': nan,
 '4HDEBRISOQUINEte': nan,
 '4HOXPACDOX_NADP': 0.0,
 '4MOPt2im': nan,
 '4MPTNLte': nan,
 '4NPHSULT': 9.65486042,
 '4PYRDX': nan,
 '5ADTSTSTERONEGLCtr': nan,
 '5ADTSTSTERONESULT': nan,
 '5ADTSTSTERONEte': nan,
 '5FTHFt2_1': 10

In [38]:

expressionrxn = model.get_reaction_scores(data_dict)

In [39]:
expressionrxn

{'FAOXC11C9m': None,
 'HMR_3686': None,
 'CBPter': None,
 'RE2069C': None,
 'OCOAT1m': None,
 'HMR_1280': None,
 'GCC2bim': None,
 'EX_retn_e': None,
 'C13856te': None,
 'ABUTD': None,
 'EX_pcholn1836_hs_e': None,
 'HMR_6672': None,
 'EX_tetdec2crn_e': None,
 'EX_fru_e': None,
 'G6PPer': None,
 'HMR_0371': None,
 'ECOAH1m': None,
 'EX_4nph_e': None,
 'r1594': None,
 'EX_C06315_e': None,
 'EX_hista_e': None,
 'r0210': None,
 'EX_HC02198_e': None,
 'IVCRNe': None,
 'NAGAlby': None,
 'EX_lcts_e': None,
 'SPHMYLN18115_hs_ATPt': None,
 'MTHFDm': None,
 'EX_chsterols_e': None,
 'HMR_7330': None,
 'EX_phacgly_e': None,
 'XYLTer': None,
 'GLYLEUHYDROc': None,
 'HMR_2852': None,
 'PCHOLN183_HSABCt': None,
 '5DHFtl': None,
 'r1842': None,
 'EX_sphmyln181181_hs_e': None,
 'HMR_0823': None,
 'RE3243C': None,
 'EX_cit_e': None,
 'EX_M00265_e': None,
 'FAOXC201C181x': None,
 'HMR_2342': None,
 'DURADx': None,
 'EX_ach_e': None,
 'RE2655R': None,
 'HMR_0873': None,
 'RE3181C': None,
 'ARGSL': None,
 

In [40]:
expressionrxn = pd.DataFrame(list(expressionrxn.items()))
expressionrxn.columns = ['', '0']

In [41]:
expressionrxn

,,0
0,FAOXC11C9m,None
1,HMR_3686,None
2,CBPter,None
3,RE2069C,None
4,OCOAT1m,None
...,...,...
6356,EX_h2co3_e,None
6357,HMR_0846,None
6358,LCAT15e,None
6359,SPHMYLN18117_hs_ATPt,None


In [13]:
expressionrxn.to_csv('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_iMAT/RXN_EXPRESSION_iMAT.csv', index=False)

In [7]:
original_data,_ = load_glial_data('/home/jiang817/cybergut_project/Draft_model/Draft_model_with_iMAT/RXN_EXPRESSION_iMAT.csv')
iMAT_data = original_data.fillna((0.2+0.5)/2)

In [8]:
protected_rxns = rx_names.index('BIOMASS_maintenance')
properties = IMATProperties(
    exp_vector = np.array(iMAT_data['0'].tolist()), # check what input to here
    exp_thresholds=(0.2,0.5),
    core=[protected_rxns]
)

method = IMAT(S, lb.astype(float), ub.astype(float), properties)

In [11]:
protected_rxns

2876

In [9]:
to_remove = np.setdiff1d(list(range(S.shape[1])), method.run())

In [10]:
len(to_remove)

4804

In [12]:
for i in to_remove:
    print(i)

0
1
6
8
9
10
11
14
15
16
17
18
19
20
21
23
27
30
31
33
34
36
37
38
39
41
43
45
49
50
51
53
54
55
56
57
58
59
60
61
62
70
72
75
76
77
81
83
84
85
86
88
89
95
99
101
104
106
109
110
114
115
116
117
118
120
123
124
125
136
137
138
142
150
151
152
156
157
159
162
163
166
170
173
175
181
185
186
187
188
189
191
192
193
194
195
197
199
200
202
204
206
207
210
217
223
224
225
231
233
234
245
246
248
249
253
255
257
269
270
271
272
273
274
276
278
281
283
285
292
300
304
311
312
313
315
331
332
334
335
336
337
339
344
345
347
348
349
353
356
359
361
365
368
373
374
375
377
378
379
380
381
382
383
384
386
387
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
409
411
412
413
414
415
416
417
418
419
420
422
423
424
425
426
427
428
429
430
431
432
433
434
437
439
440
443
444
446
448
449
453
454
458
460
465
466
467
468
470
471
473
476
477
478
479
480
481
482
483
488
491
492
495
497
498
499
500
501
506
510
517
519
522
525
526
528
529
530
531
532
534
535
536
537
538
539
542
543
545
548
549


In [ ]:
model_iMAT = model_cobra.copy()

In [ ]:
r_ids = [r.id for r in model_iMAT.reactions]
to_remove_ids = [r_ids[r] for r in to_remove]
model_iMAT.remove_reactions(to_remove_ids,True)